In [75]:
import pyqubo
from pyqubo import Array, Binary, Constraint, Placeholder
import numpy as np


def binary2integer_norm(z):
    try:
        K = len(z)
    except:
        return 1/2 * z
    return sum([2**((k+1)-1-K)*z[k] for k in range(0,K)])

N, K = 2,3
T = 2
z = 0.05
eps = 0.1

# Data variables

Sigma = np.random.rand(N, N) # UPDATE

Mu =  np.random.rand(N) # UPDATE

R = np.random.rand(T,N) # UPDATE

# Slack, nonbasic and other variables

z_full = Array.create("z_full", shape= (N,K), vartype="BINARY")

r_eps_norm = Array.create("r_eps", shape=(K, 1), vartype="BINARY")

A_norm =  Array.create("A", shape=(K, 1), vartype="BINARY")
A_max = z
A = A_max*binary2integer_norm(A_norm)

B_norm =  Array.create("B_norm", shape=(T, K), vartype="BINARY")
B_max = np.random.rand(T) # UPDATE

C_norm =  Array.create("C", shape=(K, 1), vartype="BINARY")
C = C_max*binary2integer_norm(C_norm)
C_max = eps*T

y = Array.create("y", shape=(T, 1), vartype="BINARY")

r_eps = z*(-1 + binary2integer_norm(r_eps_norm))


def IV(n):
    return (n*n - n)/2 + n

In [76]:
H_var = 0
for i in range(N):
    for j in range(N):
        H_var += binary2integer_norm(z_full[i])*binary2integer_norm(z_full[j])*Sigma[i,j]

In [77]:
H_exp = 0
for i in range(N):
        H_exp += Mu[i]*binary2integer_norm(z_full[i])

In [78]:
L1 = 100
L2s = [100]*T
L3 = 100
L4 = 10
M = 100

H1 = L1*Constraint((-r_eps[0] - z + A[0])**2, "H1")

H2 = sum([L2s[t]*Constraint((r_eps[0] 
                       - sum([R[t,j]*binary2integer_norm(z_full[j]) for j in range(N)]) 
                       - M*(1-y[t][0])+1-1 
                       + B_max[t]*binary2integer_norm(B_norm[t]))**2 , "H2_{}".format(t)) for t in range(T)])

H3 = L3*Constraint(((1-eps)*T - sum(y)[0] + C[0] )**2, "H3")

H4 = L4*Constraint((binary2integer_norm(z_full[0])-1)**2, "H4")

In [79]:

H = H_var - H_exp + (H2+1-1) + (H4+1-1) + (H3+1-1)

In [80]:
model = H.compile()

In [81]:
qubo, offset = model.to_qubo() #H_dict)

In [82]:
from dwave.samplers import SimulatedAnnealingSampler

In [83]:
len(qubo)

150

In [84]:
sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample_qubo(qubo, num_reads=10)
samples = model.decode_sampleset(sampleset)
best_sample = min(samples, key=lambda s: s.energy)
print(best_sample.sample)

{'B_norm[0][0]': 1, 'B_norm[0][1]': 1, 'B_norm[0][2]': 1, 'B_norm[1][0]': 1, 'C[0][0]': 1, 'B_norm[1][1]': 1, 'B_norm[1][2]': 1, 'C[1][0]': 1, 'C[2][0]': 1, 'r_eps[0][0]': 1, 'r_eps[1][0]': 1, 'r_eps[2][0]': 1, 'y[0][0]': 1, 'y[1][0]': 1, 'z_full[0][0]': 1, 'z_full[0][1]': 0, 'z_full[0][2]': 1, 'z_full[1][0]': 0, 'z_full[1][1]': 0, 'z_full[1][2]': 0}


In [85]:
list(best_sample.sample.keys())[0]

'B_norm[0][0]'

In [86]:
best_sample.sample

{'B_norm[0][0]': 1,
 'B_norm[0][1]': 1,
 'B_norm[0][2]': 1,
 'B_norm[1][0]': 1,
 'C[0][0]': 1,
 'B_norm[1][1]': 1,
 'B_norm[1][2]': 1,
 'C[1][0]': 1,
 'C[2][0]': 1,
 'r_eps[0][0]': 1,
 'r_eps[1][0]': 1,
 'r_eps[2][0]': 1,
 'y[0][0]': 1,
 'y[1][0]': 1,
 'z_full[0][0]': 1,
 'z_full[0][1]': 0,
 'z_full[0][2]': 1,
 'z_full[1][0]': 0,
 'z_full[1][1]': 0,
 'z_full[1][2]': 0}

In [87]:
print(best_sample.constraints())

{'H2_1': (False, 0.00011264725981163792), 'H4': (False, 0.140625), 'H3': (False, 0.0006250000000003197), 'H2_0': (False, 0.00032004287349174376)}


In [88]:
z_full

Array([[Binary('z_full[0][0]'), Binary('z_full[0][1]'), Binary('z_full[0][2]')],
       [Binary('z_full[1][0]'), Binary('z_full[1][1]'), Binary('z_full[1][2]')]])

In [73]:
z_full_array = np.zeros(shape=z_full.shape)
r_eps_norm_array = np.zeros(shape=r_eps_norm.shape)
A_norm_array = np.zeros(shape=A_norm.shape)
B_norm_array = np.zeros(shape=B_norm.shape)
C_norm_array = np.zeros(shape=C_norm.shape)
y_array = np.zeros(shape=y.shape) 
r_eps_array = np.zeros(shape=r_eps.shape)

In [72]:
import re

key = "B_full[0][1]"


data =best_sample.sample 


# Iterate through the dictionary and populate arrays
for key, value in data.items():
    if key.startswith('B_full'):
        # Use regular expression to extract indices
        indices = [int(idx) for idx in re.findall(r'\[(\d+)\]', key)]
        print(indices)
        B_norm_array[indices[0]][indices[1]] = value
    elif key.startswith('C'):
        indices = [int(idx) for idx in re.findall(r'\[(\d+)\]', key)]
        C_array[indices[0]][indices[1]] = value
    elif key.startswith('r_eps'):
        indices = [int(idx) for idx in re.findall(r'\[(\d+)\]', key)]
        r_eps_array[indices[0]][indices[1]] = value
    elif key.startswith('y'):
        indices = [int(idx) for idx in re.findall(r'\[(\d+)\]', key)]
        y_array[indices[0]][indices[1]] = value
    elif key.startswith('z_full'):
        indices = [int(idx) for idx in re.findall(r'\[(\d+)\]', key)]
        z_full_array[indices[0]][indices[1]] = value

# Print the populated arrays
print("B_full:", B_full)
print("C:", C)
print("r_eps:", r_eps)
print("y:", y)
print("z_full:", z_full)


[0, 0]
[0, 1]
[0, 2]
[1, 0]
[1, 1]
[1, 2]
B_full: [[1, 1, 1], [0, 0, 1]]
C: [[0, 0, 0], [1, 0, 0], [0, 0, 0]]
r_eps: [[1], [1], [1]]
y: [[1], [1]]
z_full: [[1, 0, 1], [0, 0, 0]]


In [55]:
import re

data = {
    'B_full[0][0]': 1,
    'B_full[0][1]': 1,
    'B_full[0][2]': 1,
    'B_full[1][0]': 0,
    'C[1][0]': 1,
    'B_full[1][1]': 0,
    'C[0][0]': 0,
    'r_eps[2][0]': 1,
    'B_full[1][2]': 1,
    'C[2][0]': 0,
    'r_eps[0][0]': 1,
    'r_eps[1][0]': 1,
    'y[0][0]': 1,
    'y[1][0]': 1,
    'z_full[0][0]': 1,
    'z_full[0][1]': 0,
    'z_full[0][2]': 1,
    'z_full[1][0]': 0,
    'z_full[1][1]': 0,
    'z_full[1][2]': 0
}

# Create arrays dynamically based on the keys
arrays = {}

# Iterate through the dictionary and populate arrays
for key, value in data.items():
    indices = [int(idx) for idx in re.findall(r'\[(\d+)\]', key)]
    
    # Extract the array name (e.g., B_full, C, r_eps, etc.)
    array_name = key.split('[')[0]

    # If the array is not in the dictionary, create an empty array
    if array_name not in arrays:
        arrays[array_name] = [[0] * (max(indices[1] + 1, 1) if len(indices) > 1 else 1) for _ in range(max(indices[0] + 1, 1))]

    # Populate the array
    arrays[array_name][indices[0]][indices[1] if len(indices) > 1 else 0] = value

# Print the populated arrays
for array_name, array in arrays.items():
    print(f"{array_name}: {array}")


IndexError: list assignment index out of range

In [57]:
def extract_dimensions(data):
    dimensions = {}

    for key in data.keys():
        # Split the key using '[' and ']' to extract indices
        indices = [int(idx) for idx in re.findall(r'\[(\d+)\]', key)]
    
        # Extract dimension name (e.g., 'B_full', 'C', 'r_eps', 'y', 'z_full')
        dimension_name = key.split('[')[0]

        # Update the dimensions dictionary
        if dimension_name not in dimensions:
            dimensions[dimension_name] = indices

    return dimensions

data = {
    'B_full[0][0]': 1,
    'B_full[0][1]': 1,
    'B_full[0][2]': 1,
    'B_full[1][0]': 0,
    'C[1][0]': 1,
    'B_full[1][1]': 0,
    'C[0][0]': 0,
    'r_eps[2][0]': 1,
    'B_full[1][2]': 1,
    'C[2][0]': 0,
    'r_eps[0][0]': 1,
    'r_eps[1][0]': 1,
    'y[0][0]': 1,
    'y[1][0]': 1,
    'z_full[0][0]': 1,
    'z_full[0][1]': 0,
    'z_full[0][2]': 1,
    'z_full[1][0]': 0,
    'z_full[1][1]': 0,
    'z_full[1][2]': 0
}

dimensions = extract_dimensions(data)
print(dimensions)


{'B_full': [0, 0], 'C': [1, 0], 'r_eps': [2, 0], 'y': [0, 0], 'z_full': [0, 0]}


In [58]:
import re

def extract_dimensions(data):
    dimensions = {}

    for key in data.keys():
        # Split the key using '[' and ']' to extract indices
        indices = [int(idx) for idx in re.findall(r'\[(\d+)\]', key)]

        # Extract dimension name (e.g., 'B_full', 'C', 'r_eps', 'y', 'z_full')
        dimension_name = key.split('[')[0]

        # Update the dimensions dictionary with the largest indices
        if dimension_name not in dimensions:
            dimensions[dimension_name] = indices
        else:
            dimensions[dimension_name] = [max(curr, new) for curr, new in zip(dimensions[dimension_name], indices)]

    return dimensions

data = {
    'B_full[0][0]': 1,
    'B_full[0][1]': 1,
    'B_full[0][2]': 1,
    'B_full[1][0]': 0,
    'C[1][0]': 1,
    'B_full[1][1]': 0,
    'C[0][0]': 0,
    'r_eps[2][0]': 1,
    'B_full[1][2]': 1,
    'C[2][0]': 0,
    'r_eps[0][0]': 1,
    'r_eps[1][0]': 1,
    'y[0][0]': 1,
    'y[1][0]': 1,
    'z_full[0][0]': 1,
    'z_full[0][1]': 0,
    'z_full[0][2]': 1,
    'z_full[1][0]': 0,
    'z_full[1][1]': 0,
    'z_full[1][2]': 0
}

dimensions = extract_dimensions(data)
print(dimensions)


{'B_full': [1, 2], 'C': [2, 0], 'r_eps': [2, 0], 'y': [1, 0], 'z_full': [1, 2]}
